In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [36]:
df_full = pd.read_json('preprocessed_text.json')

In [37]:
df_full['clean_descr'] = df_full['d_wo_sw'].apply(lambda x: ' '.join(x))

In [38]:
df = df_full[['id', 'clean_descr', 'proffesional_roles']]

In [39]:
len(df)

23771

In [40]:
not_to_delete = ['Системный аналитик' ,'Продуктовый аналитик', 'Финансовый аналитик, инвестиционный аналитик',
 'Маркетолог-аналитик', 'Бизнес-аналитик', 'BI-аналитик, аналитик данных', 'Руководитель отдела аналитики', 'Дата-сайентист']

In [41]:
df = df[df['proffesional_roles'].isin(not_to_delete)]
len(df)

8546

In [42]:
df = df.drop_duplicates(['clean_descr'])
len(df)

7420

In [43]:
df.reset_index(drop = True, inplace=True)

In [44]:
X = df['clean_descr']
y = df['proffesional_roles']

In [45]:
# Split into Training and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=df['proffesional_roles'])

In [46]:
print('Total number of descriptions: {}'.format(len(df)))
print(40*'-')
print('Split by proffesional_roles:')
print(df["proffesional_roles"].value_counts())
print(40*'-')
nr_categories = len(df["proffesional_roles"].unique())
print("Number of proffesional_roles: {n}".format(n=nr_categories))

Total number of descriptions: 7420
----------------------------------------
Split by proffesional_roles:
Системный аналитик                              1984
Бизнес-аналитик                                 1906
Финансовый аналитик, инвестиционный аналитик    1603
BI-аналитик, аналитик данных                     786
Маркетолог-аналитик                              499
Продуктовый аналитик                             395
Руководитель отдела аналитики                    217
Дата-сайентист                                    30
Name: proffesional_roles, dtype: int64
----------------------------------------
Number of proffesional_roles: 8


In [47]:
y_train.value_counts()/y.value_counts()

Системный аналитик                              0.700101
Бизнес-аналитик                                 0.699895
Финансовый аналитик, инвестиционный аналитик    0.699938
BI-аналитик, аналитик данных                    0.699746
Маркетолог-аналитик                             0.699399
Продуктовый аналитик                            0.701266
Руководитель отдела аналитики                   0.700461
Дата-сайентист                                  0.700000
Name: proffesional_roles, dtype: float64

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
corpus = X_train
# Initizalize the vectorizer with max nr words and ngrams (1: single words, 2: two words in a row)
vectorizer_tfidf = TfidfVectorizer(max_features=15000, ngram_range=(1,2))
# Fit the vectorizer to the training data
vectorizer_tfidf.fit(corpus)
TfidfVectorizer(max_features=15000, ngram_range=(1, 2))

TfidfVectorizer(max_features=15000, ngram_range=(1, 2))

In [50]:
from datetime import datetime
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [51]:
#classifier_tfidf = KNeighborsClassifier(n_neighbors=15)
classifier_tfidf = LogisticRegression()
#classifier_tfidf = MultinomialNB()
# classifier_tfidf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
model_tfidf = Pipeline([("vectorizer", vectorizer_tfidf), ("classifier", classifier_tfidf)])

start_time = datetime.now()
model_tfidf.fit(X_train, y_train)
end_time = datetime.now()

training_time_tfidf = (end_time - start_time).total_seconds()

In [52]:
from sklearn.metrics import accuracy_score

In [53]:
predicted_train_tfidf = model_tfidf.predict(X_train)
accuracy_train_tfidf = accuracy_score(y_train, predicted_train_tfidf)
print('Accuracy Training data: {:.1%}'.format(accuracy_train_tfidf))

predicted_test_tfidf = model_tfidf.predict(X_test)
accuracy_test_tfidf = accuracy_score(y_test, predicted_test_tfidf)
accuracy_tfidf = accuracy_test_tfidf
print('Accuracy Test data: {:.1%}'.format(accuracy_test_tfidf))

print('Training time: {:.1f}s'.format(training_time_tfidf))

Accuracy Training data: 88.4%
Accuracy Test data: 77.9%
Training time: 15.0s


In [54]:
print('Classes of the model: ',classifier_tfidf.classes_)
print(80*'-')
print('Shape of the coefficients of the model (categories x vocabulary size): ',classifier_tfidf.coef_.shape)
print(80*'-')
NN = 10
# Get the 10 (here: NN, which you can adjust yourself) ids of the words with highest weights per category
top_words = np.argsort(classifier_tfidf.coef_,axis=1)[:,-NN:]

# Get the vocabulary of the model (mapping of words to ids):
voc = vectorizer_tfidf.vocabulary_
# Get the inverse vocabulary to map the ids of the words to the words:
inv_voc = {v: k for k, v in voc.items()}

# Get for each category (=class) the top ten words
for n, w in enumerate(classifier_tfidf.classes_):
    t = w + ': '
    for i in range(NN):
        t += inv_voc[top_words[n,i]]
        if i!=NN:
            t+=', '
    print(t)
    print(80*'-')

Classes of the model:  ['BI-аналитик, аналитик данных' 'Бизнес-аналитик' 'Дата-сайентист'
 'Маркетолог-аналитик' 'Продуктовый аналитик'
 'Руководитель отдела аналитики' 'Системный аналитик'
 'Финансовый аналитик, инвестиционный аналитик']
--------------------------------------------------------------------------------
Shape of the coefficients of the model (categories x vocabulary size):  (8, 15000)
--------------------------------------------------------------------------------
BI-аналитик, аналитик данных: дашбордовый, dwh, power bi, power, аналитика данные, python, data, sql, bi, данные, 
--------------------------------------------------------------------------------
Бизнес-аналитик: роль бизнес, бизнес системный, требование, описание, искать бизнес, бизнес аналитик, бизнес процесс, процесс, бизнес аналитика, бизнес, 
--------------------------------------------------------------------------------
Дата-сайентист: качество данные, качество, модель машинный, обучение, данные, python,

In [55]:
df2 = df_full[['clean_descr', 'proffesional_roles']]
df2 = df2.drop_duplicates(['clean_descr'])
df2 = df2[df2['proffesional_roles'].isin(['Аналитик'])]

In [56]:
predicted_train_tfidf = model_tfidf.predict(df2['clean_descr'])
df2.reset_index(drop=True, inplace = True)

In [57]:
df2['predicted'] = pd.Series(predicted_train_tfidf)

In [58]:
df2

,clean_descr,proffesional_roles,predicted
0,ждать анализ дать производственный процесс под...,Аналитик,Бизнес-аналитик
1,компания айти консалтинг являться аккредитоват...,Аналитик,Бизнес-аналитик
2,ждать формирование развитие аналитический обес...,Аналитик,Бизнес-аналитик
3,искать ведущий системный аналитика команда раз...,Аналитик,Системный аналитик
4,тинькофф это свой экосистема инвестиция путеше...,Аналитик,"BI-аналитик, аналитик данных"
...,...,...,...
11258,компания prof it group заниматься комплексный ...,Аналитик,Бизнес-аналитик
11259,искать бизнес аналитика junior яндекс данный м...,Аналитик,Бизнес-аналитик
11260,обязанность разработка управление система расч...,Аналитик,"Финансовый аналитик, инвестиционный аналитик"
11261,responsibilities support sales amp marketing t...,Аналитик,"Финансовый аналитик, инвестиционный аналитик"
